## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-12-45-02 +0000,bbc,Work to begin on Tata's new electric furnace,https://www.bbc.com/news/articles/crl0176kz6po
1,2025-07-14-12-40-30 +0000,bbc,Argentina's Milei told to 'grow up' by VP in s...,https://www.bbc.com/news/articles/cjrlp75p8x2o
2,2025-07-14-12-37-47 +0000,nyt,It’s No Bluff: The Tariff Rate Is Soaring Unde...,https://www.nytimes.com/2025/07/14/us/politics...
3,2025-07-14-12-12-57 +0000,bbc,Hackers blamed for antisemitic posts on Elmo's...,https://www.bbc.com/news/articles/c04d25g9v6zo
4,2025-07-14-12-07-37 +0000,bbc,Trump will be hosted by King at Windsor during...,https://www.bbc.com/news/articles/c4g25ne7gw6o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,5
131,president,3
116,was,3
3,electric,2
67,between,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
28,2025-07-13-15-18-43 +0000,nyt,"To Sidestep Trump Tariffs, Asian Nations Seek ...",https://www.nytimes.com/2025/07/13/business/ec...,16
17,2025-07-13-22-59-10 +0000,bbc,"Trump, Coldplay and red carpets - was this Sup...",https://www.bbc.com/sport/football/articles/c3...,14
15,2025-07-14-01-15-53 +0000,bbc,How Trump public broadcasting cuts could hit r...,https://www.bbc.com/news/articles/c20w51rkw4no,13
20,2025-07-13-22-20-06 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o,13
21,2025-07-13-20-15-07 +0000,nyt,South African President Opens Corruption Inqui...,https://www.nytimes.com/2025/07/13/world/afric...,13


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
28,16,2025-07-13-15-18-43 +0000,nyt,"To Sidestep Trump Tariffs, Asian Nations Seek ...",https://www.nytimes.com/2025/07/13/business/ec...
20,13,2025-07-13-22-20-06 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o
11,12,2025-07-14-09-00-20 +0000,nyt,South African AIDS Activist Pushes for H.I.V. ...,https://www.nytimes.com/2025/07/14/health/sout...
22,12,2025-07-13-17-20-02 +0000,bbc,Iran president was reportedly injured in Israe...,https://www.bbc.com/news/articles/cn0z8n037p6o
27,11,2025-07-13-15-20-08 +0000,bbc,Texas camp buildings were removed from map sho...,https://www.bbc.com/news/articles/ce8zjk5yx8wo
26,11,2025-07-13-16-13-14 +0000,nyt,"In Canada’s Northern Outposts, Rusting Relics ...",https://www.nytimes.com/2025/07/13/world/ameri...
29,9,2025-07-13-14-20-15 +0000,bbc,"Electric cars will be cheaper to buy, pledges ...",https://www.bbc.com/news/articles/cg5z4nlned0o
10,9,2025-07-14-09-52-36 +0000,bbc,Clashes between Druze and Bedouin in southern ...,https://www.bbc.com/news/articles/cgmwp70pwmyo
1,8,2025-07-14-12-40-30 +0000,bbc,Argentina's Milei told to 'grow up' by VP in s...,https://www.bbc.com/news/articles/cjrlp75p8x2o
5,8,2025-07-14-11-51-13 +0000,bbc,MP facing sex assault charges given trial date,https://www.bbc.com/news/articles/c75r9wzq0l1o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
